In [1]:
import os
import requests
import json
import openai
import re
import traceback
from pprint import pprint
from IPython.display import display, JSON

# APIキー
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINATA_JWT = os.getenv("PINATA_JWT")

client = openai.OpenAI(api_key=OPENAI_API_KEY)

# API エンドポイント
FNAME_API = "https://fnames.farcaster.xyz/transfers/current?name={}"
FARCASTER_PROFILE_API = "https://nemes.farcaster.xyz:2281/v1/userDataByFid?fid={}"
FARCASTER_POSTS_API = "https://api.pinata.cloud/v3/farcaster/casts?fid={}&limit=50&order=desc"

HEADERS = {"User-Agent": "Mozilla/5.0"}

# ユーザー名
username = "rz"

In [2]:
# `username` から `FID` を取得
url = FNAME_API.format(username)
print(f"🔍 Fetching FID for username: {username}")
try:
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    data = response.json()
    fid = data.get("transfer", {}).get("to")
    print("✅ Raw FID Data:")
    display(JSON(data))  # JSONデータを表示
except Exception as e:
    print(f"❌ Error getting FID: {e}")
    fid = None

🔍 Fetching FID for username: rz
✅ Raw FID Data:


<IPython.core.display.JSON object>

In [5]:
if fid:
    # `FID` からプロフィールを取得
    url = FARCASTER_POSTS_API.format(fid)
    headers = {"accept": "application/json", "authorization": f"Bearer {PINATA_JWT}"}
    print(f"🔍 Fetching posts for FID: {fid}")
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        posts_data = response.json()
        print("✅ Raw Posts Data:")
        display(JSON(posts_data))
        
        # Bioの抽出
        author_data = posts_data.get("casts", [])[0].get("author", {})
        bio = author_data.get("bio", "No bio available.")
        print(f"📌 Extracted Bio: {bio}")
        
        # Postの抽出
        posts = [cast.get("text", "") for cast in posts_data.get("casts", []) if "text" in cast]
        posts = posts if posts else ["No posts available."]
        print("📌 Extracted Posts:")
        pprint(posts)
    except Exception as e:
        print(f"❌ Error getting posts: {e}")
        bio = "No bio available."
        posts = ["No posts available."]

    # JSONプロファイルを生成
    print(f"🔍 Generating JSON profile for: {username}")
    
    # 明示的に変数が取得できていることを確認
    print(f"🔍 DEBUG: Username: {username}")
    print(f"🔍 DEBUG: Bio: {bio}")
    print(f"🔍 DEBUG: Posts: {posts}")


🔍 Fetching posts for FID: 144
✅ Raw Posts Data:


<IPython.core.display.JSON object>

📌 Extracted Bio: cofounder @canvas, prev @common, squad @verses, selkie/acc and raymond.xyz
📌 Extracted Posts:
['Good governance: https://x.com/stopachka/status/1891225431866827156?s=46',
 'eth- (crowd cheers) cel (crowd boos) ...leration!! (crowd breaks out into '
 'applause)',
 'One angle on this that’s actually interesting is Cosmos, which never took on '
 'Ethereum’s research focus or Solana’s strategy focus, but instead had a '
 'technical fundamentals focus (paired with economic autarky)\n'
 '\n'
 'First call with ICF we talked about IBC as TCP/IP, and it was actually '
 'credible',
 'Also this',
 'I think about this all the time',
 "No one place in particular, but seems people feel like their voices aren't "
 'being heard, in Eth, in the US, etc\n'
 '\n'
 'Public figures that used to participate in good faith discourse are being '
 'replaced by people like Nikita Bier who are knowledgeable but also make '
 'numbers go up',
 'I stole this cast',
 'Governance.\n\nWe gotta figure t

In [7]:
prompt = f"""
    You are an AI assistant that generates structured JSON profiles based on user activity.

    **Rules:**
    - **MUST return JSON output ONLY** (no explanations or additional text).
    - **Infer user attributes even if data is limited**.
    - **Do not return \"Unknown\". Provide the best reasonable guess**.

    ---
    
    **User Data**
    - **Name:** {username}
    - **Bio:** {bio}
    - **Recent Posts:** {', '.join(posts)}

    ---
    
    **Output Format (JSON)**
    ```json
    {{
        "profile": {{
            "fullName": "{username}",
            "bio": "{bio}",
            "posts": {json.dumps(posts)},
            "knowledge": [
                "Extracted from bio and posts"
            ],
            "topics": [
                "Identified from user activity"
            ],
            "style": {{
                "all": ["General communication style"],
                "chat": ["How the user interacts in conversations"],
                "post": ["How the user writes posts"]
            }},
            "adjectives": [
                "Words describing the user's personality"
            ]
        }}
    }}
    ```
    """
    
print("🔍 Generated Prompt:")
print(prompt)
    
try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        json_content = response.choices[0].message.content.strip()
        print("🔍 OpenAI Response (Raw):")
        print(json_content)

        json_content = re.sub(r"```json\n|\n```", "", json_content).strip()
        json_output = json.loads(json_content)
        print(json.dumps(json_output, indent=4))
except Exception as e:
        print(f"❌ Error generating JSON: {e}")
        print(traceback.format_exc())
else:
    print(f"❌ Could not retrieve FID for username: {username}")



🔍 OpenAI Response (Raw):
```json
{
    "profile": {
        "fullName": "rz",
        "bio": "cofounder @canvas, prev @common, squad @verses, selkie/acc and raymond.xyz",
        "posts": ["Good governance: https://x.com/stopachka/status/1891225431866827156?s=46", "eth- (crowd cheers) cel (crowd boos) ...leration!! (crowd breaks out into applause)", "One angle on this that’s actually interesting is Cosmos, which never took on Ethereum’s research focus or Solana’s strategy focus, but instead had a technical fundamentals focus (paired with economic autarky)\n\nFirst call with ICF we talked about IBC as TCP/IP, and it was actually credible", "Also this", "I think about this all the time", "No one place in particular, but seems people feel like their voices aren't being heard, in Eth, in the US, etc\n\nPublic figures that used to participate in good faith discourse are being replaced by people like Nikita Bier who are knowledgeable but also make numbers go up", "I stole this cast", "Govern